<a href="https://colab.research.google.com/github/ZeleOeO/Artificial-Intelligence-Projects/blob/main/Fake_vs_Real_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m spacy download en_core_web_lg

In [2]:
import pandas as pd
url="/content/drive/MyDrive/Fake_Real_Data.csv"
df = pd.read_csv(url)
df.head()

,Text,label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake
1,U.S. conservative leader optimistic of common ...,Real
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real
3,Court Forces Ohio To Allow Millions Of Illega...,Fake
4,Democrats say Trump agrees to work on immigrat...,Real


In [3]:
df["label_num"] = df.label.map(
    {"Fake":0,
     "Real": 1}
)
df.head()

,Text,label,label_num
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake,0
1,U.S. conservative leader optimistic of common ...,Real,1
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real,1
3,Court Forces Ohio To Allow Millions Of Illega...,Fake,0
4,Democrats say Trump agrees to work on immigrat...,Real,1


In [4]:
import spacy

nlp = spacy.load("en_core_web_lg")

def preprocess(text):
  doc = nlp(text)

  filtered_tokens=[]

  for token in doc:
    if token.is_stop or token.is_punct:
      continue
    filtered_tokens.append(token.lemma_)
  
  return " ".join(filtered_tokens)

In [6]:
df["preprocessed_text"] = df.Text.apply(preprocess).apply(lambda x: nlp(x).vector)

In [7]:
df.head()

,Text,label,label_num,preprocessed_text
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake,0,"[0.23517725, 1.5265795, -2.0906563, 0.02504843..."
1,U.S. conservative leader optimistic of common ...,Real,1,"[-0.6691326, 0.62931126, -1.5525453, 1.4249555..."
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real,1,"[-1.0787294, -0.42471677, -2.4816692, 1.496829..."
3,Court Forces Ohio To Allow Millions Of Illega...,Fake,0,"[-1.5941342, 0.4442709, -1.5567913, 2.0706172,..."
4,Democrats say Trump agrees to work on immigrat...,Real,1,"[-0.6243596, 0.7249644, -2.0244102, 1.5570543,..."


In [23]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    df.preprocessed_text,
    df.label_num,
    test_size = 0.2,
    random_state = 2022
)

In [38]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

scaler = MinMaxScaler()

train_scaler = scaler.fit_transform(np.stack(x_train))
test_scaler = scaler.fit_transform(np.stack(x_test))

In [44]:
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

svc=SVC(kernel="sigmoid", gamma=1.0)
knc= KNeighborsClassifier(n_neighbors=5)
mnb = MultinomialNB()
rfc = RandomForestClassifier()

In [45]:
clfs={"SVC":svc, "KN":knc, "NB":mnb, "RF":rfc}

def train(clf, features, targets):
  clf.fit(features, targets)

def predict(clf, features):
  return clf.predict(features)

In [46]:
pred_scores_word_vectors = []

for k,v in clfs.items():
  train(v, train_scaler, y_train)
  pred = predict(v, test_scaler)
  pred_scores_word_vectors.append((k, [accuracy_score(np.stack(y_test), pred)]))

In [47]:
pred_scores_word_vectors

[('SVC', [0.5171717171717172]),
 ('KN', [0.9792929292929293]),
 ('NB', [0.952020202020202]),
 ('RF', [0.9883838383838384])]

In [53]:
pred = predict(clfs["RF"], test_scaler)
print(classification_report(np.stack(y_test), pred))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      1024
           1       0.98      0.99      0.99       956

    accuracy                           0.99      1980
   macro avg       0.99      0.99      0.99      1980
weighted avg       0.99      0.99      0.99      1980

